# Knowledge Graph Querying: Competency Questions (CQs)
This notebook shows how to query KGs to answer the CQs related to the OntoBOT ontology.

We will:
- load local RDF graphs
- Formulate SPARQL queries corresponding to CQ

#### 🔧 Setup 🔧

In [5]:
from rdflib import Graph, Namespace
from collections import defaultdict
from rdflib.namespace import RDFS, FOAF, DCTERMS

import pandas as pd

In [41]:
# Load graph
g = Graph()
g.parse("scenarios_kgs.ttl", format="ttl")

<Graph identifier=Ndb1ce972896e436c85152f4409c93e64 (<class 'rdflib.graph.Graph'>)>

In [30]:
HI = Namespace("http://www.example/w3id.org/hi-ontology#")
HI_S = Namespace("http://www.example/w3id.org/hi-ontology/scenarios/")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")

g.bind("hi", HI)
g.bind("hi-s", HI_S)
g.bind("dul", DUL)

#### 🤖 CQ 1 🤖

**Can we identify, within a given Hybrid Intelligence scenario, the participants involved in the human–AI collaboration and the roles they assume?**

In [42]:
cq1 = """
PREFIX hi: <http://www.example/w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s: <http://www.example/w3id.org/hi-ontology/scenarios/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?participantLabel ?roleLabel
WHERE {
    VALUES ?scenario { hi-s:HIScenarioPersonalAssistant }
    
    ?scenario       hi:describesUseCase     ?usecase .
    ?usecase        hi:introducesHITeam     ?team .
    ?team           hi:hasMember            ?participant .
    ?participant    hi:hasRole              ?role ;
                    rdfs:label              ?participantLabel .
    ?role           rdfs:label              ?roleLabel .
}
"""

results_cq1 = g.query(cq1)
df_cq1 = pd.DataFrame([(str(row.participantLabel), str(row.roleLabel)) for row in results_cq1],
                        columns=["Participant", "Role"])
df_cq1

,Participant,Role
0,AI Assistant,Personal Assistant System Role.
1,Human User,Human User Role.


#### 🤖 CQ 2 🤖

**What are the capabilities required by each participant to perform a specific task within a Hybrid Intelligence goal?**

In [43]:
cq2 = """
PREFIX hi: <http://www.example/w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s: <http://www.example/w3id.org/hi-ontology/scenarios/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?participantLabel ?capabilityLabel
WHERE {
    VALUES ?scenario { hi-s:HIScenarioPersonalAssistant }
    
    ?scenario       hi:describesUseCase     ?usecase .
    ?usecase        hi:introducesHITeam     ?team .
    ?team           hi:hasMember            ?participant .
    ?participant    rdfs:label              ?participantLabel ;
                    hi:hasCapability        ?capability .
    ?capability     rdfs:label              ?capabilityLabel .
}
"""

results_cq2 = g.query(cq2)
df_cq2 = pd.DataFrame([(str(row.participantLabel), str(row.capabilityLabel)) for row in results_cq2],
                        columns=["Participant", "Capability"])
df_cq2

,Participant,Capability
0,AI Assistant,Sensing Capability
1,AI Assistant,Reasoning Capability
2,AI Assistant,Explanation Capability
3,Human User,Self-Reporting Capability


#### 🤖 CQ 3 🤖

**Given an agent's capabilities, which tasks and goals can the agent be assigned to because it satisfies all required capabilities?**

In [44]:
cq3 = """
PREFIX hi: <http://www.example/w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s: <http://www.example/w3id.org/hi-ontology/scenarios/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?goalLabel ?taskLabel
WHERE {
    VALUES ?capability { hi-s:CapabilitySensing hi-s:CapabilityReasoning hi-s:CapabilityExplanation }
    
    ?scenario       hi:describesUseCase     ?usecase .
    ?usecase        hi:introducesHITeam     ?team .
    ?team           hi:hasGoal              ?goal ;
                    hi:hasMember            ?participant .
    ?goal           rdfs:label              ?goalLabel ;
                    hi:requiresTask         ?task .
    ?task           rdfs:label              ?taskLabel .
    ?participant    hi:hasCapability        ?capability .

}
"""

results_cq3 = g.query(cq3)
df_cq3 = pd.DataFrame([(str(row.goalLabel), str(row.taskLabel)) for row in results_cq3],
                        columns=["Goal", "Task"])
df_cq3

,Goal,Task
0,User Wellbeing Goal.,Data Collection Task
1,User Wellbeing Goal.,Conflict Resolution Task
2,User Wellbeing Goal.,Explanation Task


#### 🤖 CQ 4 🤖

**Which constraints influence the execution and evaluation of a given task?**

In [ ]:
cq4 = """
PREFIX hi: <http://www.example/w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s: <http://www.example/w3id.org/hi-ontology/scenarios/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?constraintLabel ?constraintComment
WHERE {
    VALUES ?task { hi-s:TaskDataCollection }
    
    ?team           hi:hasGoal              ?goal ;
                    hi:hasEthicalConstraint ?constraint .
    ?goal           rdfs:label              ?goalLabel ;
                    hi:requiresTask         ?task .
    ?task           rdfs:label              ?taskLabel .
    ?constraint     rdfs:label              ?constraintLabel ;
                    rdfs:comment            ?constraintComment .
}
"""

results_cq4 = g.query(cq4)
df_cq4 = pd.DataFrame([(str(row.constraintLabel), str(row.constraintComment))) for row in results_cq4],
                        columns=["Constraint Label ", "Constraint Comment"])
df_cq4

#### 🤖 CQ 5 🤖

**Which methods are used by agents to carry out a given task, and what are their intended outcomes?**

#### 🤖 CQ 6 🤖

**What evaluations, metrics, or experiments have been conducted to assess the performance of a Hybrid Intelligence team?**

#### 🤖 CQ 7 🤖

**Can we identify when an agent’s behavior aligns or conflicts with an ethical constraint?**

#### 🤖 CQ 8 🤖

**Can we determine whether a team collectively possesses all capabilities needed to achieve a goal?**

#### 🤖 CQ 9 🤖

**Which Hybrid Intelligence scenarios share goals, tasks, or ethical considerations, allowing knowledge to be reused?**

#### 🤖 CQ 10 🤖

**Can reasoning infer implicit relations such as team eligibility, task assignment, or ethical influence that are not explicitly stated?**